## Machine Translation :  German to English

In [1]:
# !pip install evaluate
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [34]:
import shutil
import random
import itertools
import spacy
import pandas as pd
import numpy as np
from pprint import pprint
from tqdm import tqdm , tqdm_notebook

import evaluate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

tqdm.pandas()

## Set Seed

In [4]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## Read Data

In [5]:
train_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/seq2seq/main/data/train.csv")
val_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/seq2seq/main/data/val.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/seq2seq/main/data/test.csv")

train_df.shape, val_df.shape, test_df.shape

((29000, 2), (1014, 2), (1000, 2))

In [6]:
train_df.head(3)

,en,de
0,"Two young, White males are outside near many b...",Zwei junge weiße Männer sind im Freien in der ...
1,Several men in hard hats are operating a giant...,Mehrere Männer mit Schutzhelmen bedienen ein A...
2,A little girl climbing into a wooden playhouse.,Ein kleines Mädchen klettert in ein Spielhaus ...


## Tokenizer

In [7]:
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")

In [8]:
string = "What a lovely day it is today!"

[token.text for token in nlp_en.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

## Prepare data

In [9]:
def tokenized_text(text, nlp, max_length = 1000, is_lower = True):
    if is_lower:
        text = str(text).lower()
    tokens = [token.text for token in nlp.tokenizer(text)]
    return tokens

In [10]:
train_df['en_tokens'] = train_df.en.map(lambda x: tokenized_text(x, nlp_en))
train_df['de_tokens'] = train_df.de.map(lambda x: tokenized_text(x, nlp_de))

## val
val_df['en_tokens'] = val_df.en.map(lambda x: tokenized_text(x, nlp_en))
val_df['de_tokens'] = val_df.de.map(lambda x: tokenized_text(x, nlp_de))
## test
test_df['en_tokens'] = test_df.en.map(lambda x: tokenized_text(x, nlp_en))
test_df['de_tokens'] = test_df.de.map(lambda x: tokenized_text(x, nlp_de))

In [11]:
train_df.head(3)

,en,de,en_tokens,de_tokens
0,"Two young, White males are outside near many b...",Zwei junge weiße Männer sind im Freien in der ...,"[two, young, ,, white, males, are, outside, ne...","[zwei, junge, weiße, männer, sind, im, freien,..."
1,Several men in hard hats are operating a giant...,Mehrere Männer mit Schutzhelmen bedienen ein A...,"[several, men, in, hard, hats, are, operating,...","[mehrere, männer, mit, schutzhelmen, bedienen,..."
2,A little girl climbing into a wooden playhouse.,Ein kleines Mädchen klettert in ein Spielhaus ...,"[a, little, girl, climbing, into, a, wooden, p...","[ein, kleines, mädchen, klettert, in, ein, spi..."


## Build Vocab

In [12]:
special_words = ["<unk>","<pad>", "<sos>","<eos>"]
en_words = list(set(itertools.chain.from_iterable(train_df.en_tokens.tolist())))
de_words = list(set(itertools.chain.from_iterable(train_df.de_tokens.tolist())))

en_words = special_words + en_words
de_words = special_words + de_words

en_vocab = {w:i for i,w in enumerate(en_words)}
de_vocab = {w:i for i,w in enumerate(de_words)}

UNK_ID = en_vocab['<unk>']
PAD_ID = en_vocab['<pad>']
SOS_ID = en_vocab['<sos>']
EOS_ID = en_vocab['<eos>']

print(f"en_vocab : {len(en_vocab)}")
print(f"de_vocab : {len(de_vocab)}")

en_vocab : 9795
de_vocab : 18669


## Encode text

In [13]:
def encode_text(tokens, vocab, is_add_sos = True, is_add_eos = True):
    encoded = []
    for w in tokens:
        encoded.append(vocab.get(w, UNK_ID))

    if is_add_sos:
        encoded = [SOS_ID] + encoded
    if is_add_eos:
        encoded =  encoded + [EOS_ID]
    return encoded

In [14]:
train_df['en_encoded'] = train_df.en_tokens.apply(lambda x: encode_text(x, en_vocab))
train_df['de_encoded'] = train_df.de_tokens.apply(lambda x: encode_text(x, de_vocab))

## val
val_df['en_encoded'] = val_df.en_tokens.apply(lambda x: encode_text(x, en_vocab))
val_df['de_encoded'] = val_df.de_tokens.apply(lambda x: encode_text(x, de_vocab))
## test
test_df['en_encoded'] = test_df.en_tokens.apply(lambda x: encode_text(x, en_vocab))
test_df['de_encoded'] = test_df.de_tokens.apply(lambda x: encode_text(x, de_vocab))

In [15]:
train_df.head(3)

,en,de,en_tokens,de_tokens,en_encoded,de_encoded
0,"Two young, White males are outside near many b...",Zwei junge weiße Männer sind im Freien in der ...,"[two, young, ,, white, males, are, outside, ne...","[zwei, junge, weiße, männer, sind, im, freien,...","[2, 7299, 1325, 8398, 4755, 226, 577, 8701, 14...","[2, 1495, 6694, 9780, 1638, 8224, 4642, 16038,..."
1,Several men in hard hats are operating a giant...,Mehrere Männer mit Schutzhelmen bedienen ein A...,"[several, men, in, hard, hats, are, operating,...","[mehrere, männer, mit, schutzhelmen, bedienen,...","[2, 950, 8255, 5210, 230, 1048, 577, 6942, 379...","[2, 14004, 1638, 5708, 3788, 1417, 9361, 911, ..."
2,A little girl climbing into a wooden playhouse.,Ein kleines Mädchen klettert in ein Spielhaus ...,"[a, little, girl, climbing, into, a, wooden, p...","[ein, kleines, mädchen, klettert, in, ein, spi...","[2, 3791, 1317, 4398, 6125, 1165, 3791, 7154, ...","[2, 9361, 12382, 11351, 7168, 10069, 9361, 114..."


## Data Loaders

In [16]:
train_data = train_df[['en_encoded','de_encoded']].to_dict('records')
val_data = val_df[['en_encoded','de_encoded']].to_dict('records')
test_data = test_df[['en_encoded','de_encoded']].to_dict('records')

In [17]:
pprint(train_data[:2], compact=True)

[{'de_encoded': [2, 1495, 6694, 9780, 1638, 8224, 4642, 16038, 10069, 4241,
                 15608, 4777, 13707, 901, 3],
  'en_encoded': [2, 7299, 1325, 8398, 4755, 226, 577, 8701, 1462, 3851, 6981,
                 483, 3]},
 {'de_encoded': [2, 14004, 1638, 5708, 3788, 1417, 9361, 911, 901, 3],
  'en_encoded': [2, 950, 8255, 5210, 230, 1048, 577, 6942, 3791, 4323, 8692,
                 2792, 483, 3]}]


In [18]:
def custom_collate(batch):
    """
    Dynamic padding : find the max len in the batch and do the padding
    """
    en_batch = [torch.tensor(item['en_encoded']) for item in batch]
    de_batch = [torch.tensor(item['de_encoded']) for item in batch]

    padded_en = nn.utils.rnn.pad_sequence(en_batch, batch_first= True, padding_value= PAD_ID)
    padded_de = nn.utils.rnn.pad_sequence(de_batch, batch_first= True, padding_value= PAD_ID)

    return {"padded_en": padded_en, "padded_de":  padded_de}

In [19]:
batch_size = 3
train_dl = DataLoader(train_data , batch_size = batch_size, shuffle = False, collate_fn= custom_collate)
example = next(iter(train_dl))
padded_en, padded_de = example['padded_en'],example['padded_de']
print(padded_en.shape, padded_de.shape)

torch.Size([3, 14]) torch.Size([3, 15])


In [20]:
padded_en

tensor([[   2, 7299, 1325, 8398, 4755,  226,  577, 8701, 1462, 3851, 6981,  483,
            3,    1],
        [   2,  950, 8255, 5210,  230, 1048,  577, 6942, 3791, 4323, 8692, 2792,
          483,    3],
        [   2, 3791, 1317, 4398, 6125, 1165, 3791, 7154, 3533,  483,    3,    1,
            1,    1]])

In [21]:
padded_de

tensor([[    2,  1495,  6694,  9780,  1638,  8224,  4642, 16038, 10069,  4241,
         15608,  4777, 13707,   901,     3],
        [    2, 14004,  1638,  5708,  3788,  1417,  9361,   911,   901,     3,
             1,     1,     1,     1,     1],
        [    2,  9361, 12382, 11351,  7168, 10069,  9361, 11478,  7174,   980,
           901,     3,     1,     1,     1]])

In [22]:
## dataloaders
batch_size = 256
train_dl = DataLoader(train_data , batch_size = batch_size, shuffle = True, collate_fn= custom_collate)
val_dl = DataLoader(val_data , batch_size = batch_size, shuffle = False, collate_fn= custom_collate)
test_dl = DataLoader(test_data , batch_size = batch_size, shuffle = False, collate_fn= custom_collate)

## Building Model

In [23]:
class Encoder(nn.Module):
    """
    Encode German text
    """
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first= True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, batch):
        # batch = [batch size, sent len]
        embedded = self.dropout(self.embedding(batch))
        # embedded = [batch size, sent len, embedding dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [batch size, sent len, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [24]:
## Enoder
print(f"Input : {padded_de.shape}")
encoder = Encoder(vocab_size = len(de_vocab), emb_dim = 100, hidden_dim=64, n_layers=2, dropout=0.1)
hidden, cell = encoder(padded_de)
print(hidden.shape, cell.shape)

Input : torch.Size([3, 15])
torch.Size([2, 3, 64]) torch.Size([2, 3, 64])


In [25]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout,  batch_first= True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(1)
        # input = [batch size, 1]
        embedded = self.dropout(self.embedding(input))
        # print(embedded.shape)
        # embedded = [batch size, 1, embedding dim]
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # print(output.shape)
        # print(hidden.shape)
        # output = [batch size, seq length, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [batch size,1,hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [26]:
## Decoder
## de[:,0] -> first input
print(f"Input Hidden : {hidden.shape}")
decoder = Decoder(vocab_size = len(en_vocab), emb_dim = 100, hidden_dim=64, n_layers=2, dropout=0.1)
prediction, hidden, cell = decoder(padded_en[:,0], hidden, cell)
prediction.shape, hidden.shape, cell.shape

Input Hidden : torch.Size([2, 3, 64])


(torch.Size([3, 1, 9795]), torch.Size([2, 3, 64]), torch.Size([2, 3, 64]))

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [batch size, sent len]
        # trg = [batch size, sent len]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[0]
        trg_length = trg.shape[1]
        trg_vocab_size = self.decoder.vocab_size
        # tensor to store decoder outputs
        outputs = torch.zeros(batch_size,trg_length,trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[:,0]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            output = output.squeeze(dim = 1)
            outputs[:,t,:] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[:,t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [28]:
model = Seq2Seq(encoder, decoder, device = "cpu")
outputs = model(padded_de, padded_en, teacher_forcing_ratio = True)
print(outputs.shape)

torch.Size([3, 14, 9795])


## Training

In [29]:
class Seq2SeqLightningModel(pl.LightningModule):

    def __init__(self, encode, decoder, learning_rate, device, teacher_forcing_ratio):
        super().__init__()
        self.learning_rate = learning_rate
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.encoder = encoder
        self.decoder = decoder
        self.seq2seq_model = Seq2Seq(self.encoder, self.decoder, device)
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_ID)
        self.init_weights()

        self.train_loss = []
        self.val_loss = []
        self.test_loss = []
    
    def init_weights(self):
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -0.08, 0.08)

    def forward(self, de, en):
        logits = self.seq2seq_model(de, en, teacher_forcing_ratio = self.teacher_forcing_ratio)
        return logits

    def training_step(self, batch):
        src, trg = batch['padded_de'], batch['padded_en']
        output = self(src, trg)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        # print(output.shape, trg.shape)
        loss = self.loss_fn(output, trg)
        self.train_loss.append(loss.item())
        self.log_dict({"train_loss": loss}, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch):
        src, trg = batch['padded_de'], batch['padded_en']
        output = self(src, trg)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = self.loss_fn(output, trg)
        self.val_loss.append(loss.item())
        self.log_dict({"val_loss": loss}, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self):
        print(f"Epoch : {self.current_epoch}  \
              Train Loss : {np.mean(self.train_loss)} \
              Val Loss : {np.mean(self.val_loss)} \
              Train PPL : {np.exp(np.mean(self.train_loss))} \
              Val PPL : {np.exp(np.mean(self.val_loss))} ")

        self.train_loss =[]
        self.val_loss =[]

    
    def test_step(self, batch):
        src, trg = batch['padded_de'], batch['padded_en']
        output = self(src, trg)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = self.loss_fn(output, trg)
        self.test_loss.append(loss.item())
        return loss

    def on_test_epoch_end(self) -> None:
        print(f"Test Loss : {np.mean(self.test_loss)}  Test PPL : {np.exp(np.mean(self.test_loss))}")
        self.test_loss = []

    def configure_optimizers(self):
        opt = optim.Adam(self.parameters(), lr = self.learning_rate)
        return opt

In [30]:
## test architecture
encoder = Encoder(vocab_size = len(de_vocab), emb_dim = 256, hidden_dim=512, n_layers=2, dropout=0.5)
decoder = Decoder(vocab_size = len(en_vocab), emb_dim = 256, hidden_dim=512, n_layers=2, dropout=0.5)
model = Seq2SeqLightningModel(encoder, decoder, learning_rate= .001, device ="cpu", teacher_forcing_ratio=0.5)
outputs = model(padded_de, padded_en)
outputs.shape

torch.Size([3, 14, 9795])

In [37]:
## clear old checkpoints
checkpoints_dir = "checkpoints_logs"
if os.path.exists(checkpoints_dir):
    shutil.rmtree(checkpoints_dir)


## Model Training
encoder = Encoder(vocab_size = len(de_vocab), emb_dim = 300, hidden_dim=512, n_layers=2, dropout=0.5)
decoder = Decoder(vocab_size = len(en_vocab), emb_dim = 300, hidden_dim=512, n_layers=2, dropout=0.5)
model = Seq2SeqLightningModel(encoder, decoder, learning_rate= .001, device ="cuda", teacher_forcing_ratio=0.5)

callbacks = pl.callbacks.ModelCheckpoint(dirpath = checkpoints_dir,
                                         filename = '{epoch}-{val_loss:.2f}',
                                          mode = "min",
                                          monitor = "val_loss",
                                          save_last = True,
                                          save_top_k=-1)


trainer = pl.Trainer(accelerator= "gpu",
           max_epochs=10,
           check_val_every_n_epoch = 2,
           gradient_clip_val=1, 
           gradient_clip_algorithm="value",
           callbacks = [callbacks])

trainer.fit(model, train_dl, val_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | Encoder          | 9.4 M 
1 | decoder       | Decoder          | 11.7 M
2 | seq2seq_model | Seq2Seq          | 21.1 M
3 | loss_fn       |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch : 0                Train Loss : nan               Val Loss : 9.208253860473633               Train PPL : nan               Val PPL : 9979.156637494161 


/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 1                Train Loss : 5.398061848523324               Val Loss : 4.96839964389801               Train PPL : 220.97771261212498               Val PPL : 143.79657740100245 


In [ ]:
## if test dataset has loss near around val loss that means we are not overfitting
model = model.eval()
trainer.test(model, dataloaders= test_dl)

## Predict

In [ ]:
en_vocab_lookup = {i:w for w,i in en_vocab.items()}

def encode_tokens(tokens, vocab):
    encoded = []
    for w in tokens:
        encoded.append(vocab.get(w, UNK_ID))
    return encoded


def decode_tokens(tokens, vocab_loopup):
    decoded = []
    for i in tokens:
        if i == 2 or i==3:
            continue
        decoded.append(vocab_loopup.get(i))
    return decoded 


def translate_sentence(sentence, model, en_nlp, de_nlp, en_vocab, de_vocab, sos_token, eos_token, device, lower = True, max_output_length=15):
    ## model eval mode
    model.eval()

    model = model.to(device)
    
    ## encode german sent
    sent = str(sentence).lower() if lower else sentence
    tokens = [token.text for token in de_nlp.tokenizer(sent)]
    tokens = [sos_token] + tokens + [eos_token]
    ids = encode_tokens(tokens, de_vocab)

    ## encoder input 
    tensor = torch.LongTensor(ids).unsqueeze(0).to(device)
    tensor = tensor.to(device)

    ## encoder
    hidden, cell = model.encoder(tensor)

    ## input to decoder <sos>
    inputs = encode_tokens([sos_token], en_vocab)
    
    for _ in range(max_output_length):
        inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
        output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
        predicted_token = output.argmax(-1).item()
        inputs.append(predicted_token)
        if predicted_token == en_vocab[eos_token]:
            break

    tokens = decode_tokens(inputs, en_vocab_lookup)

    return tokens

In [ ]:
sent = "Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt."
predictions = translate_sentence(sent, model, nlp_en, nlp_de, en_vocab, de_vocab, sos_token="<sos>", eos_token="<eos>", device = "cuda")
print(predictions)

## Blue Scroe

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
prediction_list = []
prediction_list = [translate_sentence(sent, model, nlp_en, nlp_de, en_vocab, de_vocab, sos_token="<sos>", eos_token="<eos>", device = "cuda") for sent in tqdm_notebook(test_df.de.values)]
predictions = list(map(lambda x: " ".join(x),prediction_list))

In [ ]:
pprint(predictions[:2], compact = True)

In [ ]:
references = test_df.en.tolist()
pprint(references[:2], compact=True)

In [ ]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

tokenizer_fn = get_tokenizer_fn(nlp_en,  lower = True)

In [ ]:
results = bleu.compute(predictions=predictions, references=references, tokenizer=tokenizer_fn)
results